In [8]:
!pip install -q streamlit scikit-learn pandas numpy sentence-transformers nltk
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋
changed 22 packages in 2s
⠋
⠋3 packages are looking for funding
⠋  run `npm fund` for details
⠋

In [9]:
preprocess_code = '''
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# Contraction mapping
CONTRACTIONS = {
    "don't": "do not", "can't": "cannot", "won't": "will not",
    "i'm": "i am", "it's": "it is", "you're": "you are",
    "they're": "they are", "we're": "we are", "isn't": "is not",
    "aren't": "are not", "wasn't": "was not", "weren't": "were not",
    "doesn't": "does not", "didn't": "did not", "hasn't": "has not",
    "haven't": "have not", "hadn't": "had not", "couldn't": "could not",
    "wouldn't": "would not", "shouldn't": "should not", "mightn't": "might not",
    "mustn't": "must not"
}

def clean_text(text: str) -> str:
    if not text:
        return ""
    text = text.lower()
    text = re.sub(r"http\\S+|www\\S+|https\\S+", "", text)
    text = re.sub(r"\\S+@\\S+", "", text)
    for c, f in CONTRACTIONS.items():
        text = text.replace(c, f)
    text = re.sub(r"[^a-z\\s]", "", text)
    text = re.sub(r"\\s+", " ", text).strip()
    tokens = [lemmatizer.lemmatize(w) for w in text.split() if w not in stop_words]
    return " ".join(tokens)
'''
with open("preprocessing.py", "w") as f:
    f.write(preprocess_code)


In [10]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sentence_transformers import SentenceTransformer
from google.colab import files
from preprocessing import clean_text  # unified import

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [11]:
uploaded = files.upload()
import io
df = pd.read_csv(io.BytesIO(next(iter(uploaded.values()))), sep="\t")

Saving amazon_alexa.tsv to amazon_alexa (3).tsv


In [13]:
def clean_text(text):
    # Handle NaN or None values
    if pd.isnull(text):
        return ""

    # Lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)

    # Remove emails
    text = re.sub(r"\S+@\S+", "", text)

    # Expand contractions
    contractions = {
        "don't": "do not", "can't": "cannot", "won't": "will not",
        "i'm": "i am", "it's": "it is", "you're": "you are",
        "they're": "they are", "we're": "we are", "isn't": "is not",
        "aren't": "are not", "wasn't": "was not", "weren't": "were not",
        "doesn't": "does not", "didn't": "did not", "hasn't": "has not",
        "haven't": "have not", "hadn't": "had not", "couldn't": "could not",
        "wouldn't": "would not", "shouldn't": "should not", "mightn't": "might not",
        "mustn't": "must not"
    }
    for c, f in contractions.items():
        text = text.replace(c, f)

    # Remove non-alphabetic characters
    text = re.sub(r"[^a-z\s]", " ", text)

    # Remove extra spaces
    text = re.sub(r"\s+", " ", text).strip()

    # Tokenize, remove stopwords, and lemmatize
    tokens = [lemmatizer.lemmatize(w) for w in text.split() if w not in stop_words]

    return " ".join(tokens)

# Apply preprocessing safely
df["verified_reviews"] = df["verified_reviews"].apply(clean_text)

# Filter empty reviews
df = df[df["verified_reviews"].str.strip() != ""]

# Keep only 'positive' and 'negative'
df = df[df["feedback"].isin(["positive", "negative"])]

# Map labels and invert as per your code
df["feedback"] = df["feedback"].map({"positive": 1, "negative": 0}).astype(int)
y = 1 - df["feedback"]


In [14]:
embedder = SentenceTransformer("all-MiniLM-L6-v2")
X_embed = embedder.encode(df["verified_reviews"].tolist(), show_progress_bar=True)

Batches: 0it [00:00, ?it/s]

In [19]:
# Inspect raw feedback values before processing
print("Raw feedback unique values:", df["feedback"].unique())

# Normalize labels to lowercase strings
df["feedback"] = df["feedback"].astype(str).str.lower().str.strip()

# Show again after lowercase/strip
print("Normalized feedback values:", df["feedback"].unique())

# Create label mapping dynamically based on actual dataset contents
positive_keywords = ["positive", "pos", "1", "yes", "true"]
negative_keywords = ["negative", "neg", "0", "no", "false"]

label_map = {}
for val in df["feedback"].unique():
    if any(k in val for k in positive_keywords):
        label_map[val] = 1
    elif any(k in val for k in negative_keywords):
        label_map[val] = 0

print("Generated label_map:", label_map)

# Apply mapping
df["feedback"] = df["feedback"].map(label_map)

# Keep only valid rows
df = df[df["feedback"].isin([0, 1])]

# Fill NaN reviews and clean text
df["verified_reviews"] = df["verified_reviews"].fillna("").apply(clean_text)
df = df[df["verified_reviews"].str.strip() != ""]

# Target variable (invert feedback to match original code)
y = 1 - df["feedback"]

print(f"Samples remaining after cleaning: {len(df)}")
print("Unique feedback values after mapping:", df["feedback"].unique())


Raw feedback unique values: []
Normalized feedback values: []
Generated label_map: {}
Samples remaining after cleaning: 0
Unique feedback values after mapping: []


In [36]:
import os
import pickle
import pandas as pd
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer

# 0️⃣ Remove old files if they exist
for file in ["model.pkl", "embedder.pkl"]:
    if os.path.exists(file):
        os.remove(file)

# 1️⃣ Load dataset
df = pd.read_csv("amazon_alexa.tsv", sep="\t")

# 2️⃣ Simple text cleaning
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r"[^a-z0-9\s]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

df["verified_reviews"] = df["verified_reviews"].fillna("").apply(clean_text)

# 3️⃣ Remove empty reviews
df = df[df["verified_reviews"].str.strip() != ""]

# 4️⃣ Define features and labels
y = df["feedback"]  # keep original labels

# 5️⃣ Encode text to embeddings
model_embed = SentenceTransformer("all-MiniLM-L6-v2")
X_embed = model_embed.encode(df["verified_reviews"].tolist())

# 6️⃣ Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_embed, y, test_size=0.2, stratify=y, random_state=42
)

# 7️⃣ Train model
model = RandomForestClassifier(class_weight="balanced", random_state=42)
model.fit(X_train, y_train)

# 8️⃣ Save model and embedder
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

with open("embedder.pkl", "wb") as f:
    pickle.dump(model_embed, f)

# 9️⃣ Reload to verify
with open("model.pkl", "rb") as f:
    model = pickle.load(f)

with open("embedder.pkl", "rb") as f:
    model_embed = pickle.load(f)

print("✅ Model and embedder saved successfully!")


✅ Model and embedder saved successfully!


In [26]:
import os

for file in ["model.pkl", "embedder.pkl"]:
    if os.path.exists(file):
        os.remove(file)


In [35]:
from google.colab import files
uploaded = files.upload()

import pandas as pd
df_raw = pd.read_csv("amazon_alexa.tsv", sep="\t")  # specify tab separator
print(df_raw.shape)
print(df_raw.head())


Saving amazon_alexa.tsv to amazon_alexa (7).tsv
(3150, 5)
   rating       date         variation  \
0       5  31-Jul-18  Charcoal Fabric    
1       5  31-Jul-18  Charcoal Fabric    
2       4  31-Jul-18    Walnut Finish    
3       5  31-Jul-18  Charcoal Fabric    
4       5  31-Jul-18  Charcoal Fabric    

                                    verified_reviews  feedback  
0                                      Love my Echo!         1  
1                                          Loved it!         1  
2  Sometimes while playing a game, you can answer...         1  
3  I have had a lot of fun with this thing. My 4 ...         1  
4                                              Music         1  


In [40]:
!curl https://loca.lt/mytunnelpassword


35.239.120.191

In [42]:
app_code = '''
import streamlit as st
import pickle
from sentence_transformers import SentenceTransformer
import re

# Simple text cleaning
def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", "", text)
    return text.strip()

@st.cache_resource
def load_model():
    with open("model.pkl", "rb") as f:
        model = pickle.load(f)
    with open("embedder.pkl", "rb") as f:
        embedder = pickle.load(f)
    return model, embedder

model, embedder = load_model()

st.title("Amazon Alexa Review Sentiment Analysis")
st.write("Predict whether a review is **Positive** or **Negative**.")

user_input = st.text_area("Enter your review:")

if st.button("Predict"):
    cleaned = clean_text(user_input)
    if cleaned == "":
        st.warning("Please enter valid text.")
    else:
        emb = embedder.encode([cleaned])
        pred = model.predict(emb)
        sentiment = "Positive" if pred[0] == 1 else "Negative"
        st.success(f"Prediction: **{sentiment}**")
'''

with open("app.py", "w") as f:
    f.write(app_code)

# Run Streamlit in background
!nohup streamlit run app.py --server.port 8501 &>/tmp/logs.txt &

# Expose via localtunnel (no password)
!npx localtunnel --port 8501


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸your url is: https://ten-wasps-rest.loca.lt
^C
